In [1]:
from pyspark.sql import SparkSession
from math import radians, cos, sin, asin, sqrt
import pandas as pd
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import os
import sys
import matplotlib.pyplot as plt

In [2]:
# Sets up the spark session and has magic code that makes things not break
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
spark = SparkSession.builder.appName('hw5').getOrCreate()

In [3]:
# Creates a haversine function
def haversine(lat1, lon1, lat2, lon2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r

# Initiailizes the udf version of the haversine function
haversineUDF = udf(lambda x, y: haversine(28.396837, -80.605659, x, y), FloatType())

In [4]:
# Loads the stations from the csv
stations = spark.read.csv('stations.csv', header=False, inferSchema=True).withColumnRenamed("_c0", "StationID").withColumnRenamed("_c1", "WBANID").withColumnRenamed("_c2", "GPSLatitude").withColumnRenamed("_c3", "GPSLongitude")

# Creates a df with the distance from Cape Canaveral using the haversine function
stations = stations.withColumn("distance", haversineUDF(stations.GPSLatitude, stations.GPSLongitude))

In [5]:
stations.createOrReplaceTempView('stations')

In [6]:
# Removes stations that have null coordinates or IDs, removes stations that are more than 100km away from Cape Canaveral
# and drops duplicates
stations = spark.sql("""
    SELECT * FROM STATIONS
    WHERE GPSLatitude != 0 AND GPSLongitude != 0 AND distance <= 100 AND StationID IS NOT NULL
""")
stations = stations.dropDuplicates(["StationID"])
stations.show()

+---------+------+-----------+------------+---------+
|StationID|WBANID|GPSLatitude|GPSLongitude| distance|
+---------+------+-----------+------------+---------+
|   722050| 12815|     28.434|     -81.325| 70.47143|
|   722053| 12841|     28.545|     -81.333| 72.97912|
|   747946| 12886|     28.617|     -80.683|25.620928|
|   747950| 12867|     28.233|       -80.6|18.226263|
|   722040| 12838|     28.101|     -80.644|33.109257|
|   749047|  null|     28.283|     -81.416|    80.31|
|   997806|  null|       28.4|     -80.533|7.1157584|
|   720904|   299|     29.067|     -81.283| 99.57265|
|   722056| 12834|     29.183|     -81.048| 97.46734|
|   722051| 12841|     28.545|     -81.333| 72.97912|
|   747945|  null|     28.617|       -80.7|  26.1591|
|   998275|  null|     28.017|     -80.683|  42.9105|
|   747870| 12834|     29.183|     -81.048| 97.46734|
|   997354|  null|      28.42|      -80.58|3.5960674|
|   995450|  null|     28.519|     -80.166| 45.07606|
|   722046| 12898|     28.51

In [7]:
data_1986 = spark.read.csv('1986.csv', header=False, inferSchema=True).withColumnRenamed("_c0", "StationID").withColumnRenamed("_c1", "WBANID").withColumnRenamed("_c2", "Month").withColumnRenamed("_c3", "Day").withColumnRenamed("_c4", "temp")
data_1986.show()

+---------+------+-----+---+----+
|StationID|WBANID|Month|Day|temp|
+---------+------+-----+---+----+
|    10010|  null|    1|  1|17.2|
|    10010|  null|    1|  2|12.1|
|    10010|  null|    1|  3|10.4|
|    10010|  null|    1|  4|17.4|
|    10010|  null|    1|  5|26.5|
|    10010|  null|    1|  6|30.1|
|    10010|  null|    1|  7|29.7|
|    10010|  null|    1|  8|29.6|
|    10010|  null|    1|  9|29.6|
|    10010|  null|    1| 10|33.0|
|    10010|  null|    1| 11|32.5|
|    10010|  null|    1| 12|27.4|
|    10010|  null|    1| 13|22.2|
|    10010|  null|    1| 14|11.3|
|    10010|  null|    1| 15| 2.5|
|    10010|  null|    1| 16| 3.0|
|    10010|  null|    1| 17|13.4|
|    10010|  null|    1| 18|29.8|
|    10010|  null|    1| 19|27.5|
|    10010|  null|    1| 20|25.2|
+---------+------+-----+---+----+
only showing top 20 rows



In [8]:
data_1986.createOrReplaceTempView('_1986')

In [9]:
jan_1986 = spark.sql("""
    SELECT * FROM _1986
    WHERE Month == 1
""")
jan_1986.show()

+---------+------+-----+---+----+
|StationID|WBANID|Month|Day|temp|
+---------+------+-----+---+----+
|    10010|  null|    1|  1|17.2|
|    10010|  null|    1|  2|12.1|
|    10010|  null|    1|  3|10.4|
|    10010|  null|    1|  4|17.4|
|    10010|  null|    1|  5|26.5|
|    10010|  null|    1|  6|30.1|
|    10010|  null|    1|  7|29.7|
|    10010|  null|    1|  8|29.6|
|    10010|  null|    1|  9|29.6|
|    10010|  null|    1| 10|33.0|
|    10010|  null|    1| 11|32.5|
|    10010|  null|    1| 12|27.4|
|    10010|  null|    1| 13|22.2|
|    10010|  null|    1| 14|11.3|
|    10010|  null|    1| 15| 2.5|
|    10010|  null|    1| 16| 3.0|
|    10010|  null|    1| 17|13.4|
|    10010|  null|    1| 18|29.8|
|    10010|  null|    1| 19|27.5|
|    10010|  null|    1| 20|25.2|
+---------+------+-----+---+----+
only showing top 20 rows

